<a href="https://colab.research.google.com/github/yuhui-0611/ESAA/blob/main/ESAA_YB_WEEK13_2_Code_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **식음업장 메뉴 수요 예측 AI 오프라인 해커톤**

[link text](https://dacon.io/competitions/official/236594/codeshare/12883?page=1&dtype=recent)

# Import

In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm

# Fixed RandomSeed & Setting Hyperparameter

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 64, 50
LR = 1e-3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("DEVICE:", DEVICE)

# Data Load

In [ ]:
BASE_DIR = "./"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

train = pd.read_csv(os.path.join(TRAIN_DIR, "train.csv"), parse_dates=["영업일자"])

# 메타 경로
TRAIN_META_DIR = os.path.join(TRAIN_DIR, "meta")
TEST_META_DIR  = os.path.join(TEST_DIR, "meta")

# Data Preprocessing

In [ ]:
# [메타 데이터 IO/병합]
META_OPTIONS = {
    "ski":     {"use": True,  "cols": ["1일내장객"]},
    "room":    {"use": True,  "cols": None},
    "hwadam":  {"use": True,  "cols": None},
    "weather": {"use": True,  "cols": None},
    "price":   {"use": True,  "cols": ["평균판매금액"]},
}

def _read_if_exists(path, parse_dates=None):
    import pandas as pd, os
    if os.path.exists(path):
        return pd.read_csv(path, parse_dates=parse_dates)
    return pd.DataFrame()

def _normalize_weather_cols(df):
    """TRAIN_weather / TEST_weather 파일의 '일시'를 '영업일자'로 바꾸고
       날짜만 남겨 일(day) 단위로 조인되게 정규화."""
    if df is None or df.empty:
        return df
    if '일시' in df.columns:
        df = df.copy()
        df['영업일자'] = pd.to_datetime(df['일시']).dt.date
        df.drop(columns=['일시'], inplace=True)
        # 다시 Timestamp 컬럼으로 맞춤 (train/test의 '영업일자'가 Timestamp라면)
        df['영업일자'] = pd.to_datetime(df['영업일자'])
    return df

def read_train_meta(train_meta_dir, train_dir):
    ski = _read_if_exists(os.path.join(train_meta_dir, "TRAIN_ski.csv"),     parse_dates=["영업일자"])
    room= _read_if_exists(os.path.join(train_meta_dir, "TRAIN_room.csv"),    parse_dates=["영업일자"])
    hw  = _read_if_exists(os.path.join(train_meta_dir, "TRAIN_hwadam.csv"),  parse_dates=["영업일자"])
    w   = _read_if_exists(os.path.join(train_meta_dir, "TRAIN_weather.csv"), parse_dates=["일시"])
    w   = _normalize_weather_cols(w)

    price = _read_if_exists(os.path.join(train_dir, "price.csv"))
    return ski, room, hw, w, price


def read_test_meta(test_meta_dir, test_prefix):
    suffix = test_prefix.split('_')[-1]
    ski = _read_if_exists(os.path.join(test_meta_dir, f"TEST_ski_{suffix}.csv"),     parse_dates=["영업일자"])
    room= _read_if_exists(os.path.join(test_meta_dir, f"TEST_room_{suffix}.csv"),    parse_dates=["영업일자"])
    hw  = _read_if_exists(os.path.join(test_meta_dir, f"TEST_hwadam_{suffix}.csv"),  parse_dates=["영업일자"])
    w   = _read_if_exists(os.path.join(test_meta_dir, f"TEST_weather_{suffix}.csv"), parse_dates=["일시"])
    w   = _normalize_weather_cols(w)
    return ski, room, hw, w


def merge_meta(df, ski, room, hwadam, weather, price, options=None):
    if options is None:
        options = META_OPTIONS
    out = df.copy()

    def _merge_numeric_by_date(base, meta_df, allowed_cols=None):
        if meta_df is None or meta_df.empty:
            return base
        meta_df = meta_df.copy()

        # 1) 후보 컬럼 선택: allowed_cols가 주어지면 그 교집합만, 아니면 모든 숫자 컬럼
        if allowed_cols:
            num_cols = [c for c in allowed_cols if c in meta_df.columns]
        else:
            num_cols = [c for c in meta_df.columns
                        if c != "영업일자" and np.issubdtype(meta_df[c].dtype, np.number)]

        if not num_cols:
            return base

        # 2) 숫자 변환(문자 숫자 방지), 결측 처리
        meta_df[num_cols] = meta_df[num_cols].apply(pd.to_numeric, errors="coerce").fillna(0)

        return base.merge(meta_df[["영업일자"] + num_cols], on="영업일자", how="left")

    # 날짜 기반 메타 병합
    if options.get("ski", {}).get("use", False):
        out = _merge_numeric_by_date(out, ski,     options.get("ski", {}).get("cols"))
    if options.get("room", {}).get("use", False):
        out = _merge_numeric_by_date(out, room,    options.get("room", {}).get("cols"))
    if options.get("hwadam", {}).get("use", False):
        out = _merge_numeric_by_date(out, hwadam,  options.get("hwadam", {}).get("cols"))
    if options.get("weather", {}).get("use", False):
        out = _merge_numeric_by_date(out, weather, options.get("weather", {}).get("cols"))

    # price
    if options.get("price", {}).get("use", False) and price is not None and not price.empty:
        keep_cols = options["price"].get("cols") or [c for c in price.columns if c != "영업장명_메뉴명"]
        cols = ["영업장명_메뉴명"] + [c for c in keep_cols if c in price.columns and c != "영업장명_메뉴명"]
        out = out.merge(price[cols], on="영업장명_메뉴명", how="left")

    out = out.sort_values(["영업장명_메뉴명", "영업일자"]).reset_index(drop=True)
    out.fillna(0, inplace=True)
    return out


def select_meta_columns(df) :
    exclude = {"영업일자", "영업장명_메뉴명", "매출수량"}
    numeric_cols = [c for c in df.columns if c not in exclude and np.issubdtype(df[c].dtype, np.number)]
    return numeric_cols

# Define Model

In [ ]:
class MultiOutputLSTMWithMeta(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=7, meta_dim=0):
        super().__init__()
        self.use_meta = meta_dim > 0
        self.lstm = nn.LSTM(input_dim + meta_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, meta=None):
        if self.use_meta and meta is not None:
            x = torch.cat([x, meta], dim=-1)   # (B,L,1+M)
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])         # (B,H)

# Train

In [ ]:
def train_lstm_meta(train_df, train_meta_dir):
    trained_models = {}
    ski, room, hw, w, price = read_train_meta(train_meta_dir, TRAIN_DIR)

    merged = merge_meta(train_df, ski, room, hw, w, price, options=META_OPTIONS)
    meta_cols_all = select_meta_columns(merged)

    for store_menu, group in tqdm(merged.groupby(['영업장명_메뉴명']), desc='Training LSTM+Meta'):
        key = store_menu[0] if isinstance(store_menu, tuple) else store_menu
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        sales_scaler = MinMaxScaler()
        store_train['매출수량'] = sales_scaler.fit_transform(
            store_train[['매출수량']].values
        )  # <- np.ndarray

        meta_scaler = None
        if meta_cols_all:
            meta_scaler = MinMaxScaler()
            store_train[meta_cols_all] = meta_scaler.fit_transform(
                store_train[meta_cols_all].values
            )  # <- np.ndarray

        # 시퀀스 구성
        X_train, M_train, y_train = [], [], []
        sales_vals = store_train[['매출수량']].values  # (N,1) np.ndarray
        meta_vals  = store_train[meta_cols_all].values if meta_cols_all else None

        for i in range(len(sales_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(sales_vals[i:i+LOOKBACK])
            if meta_vals is not None:
                M_train.append(meta_vals[i:i+LOOKBACK])
            y_train.append(sales_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = torch.tensor(np.array(X_train)).float().to(DEVICE)
        y_train = torch.tensor(np.array(y_train)).float().to(DEVICE)
        M_train = torch.tensor(np.array(M_train)).float().to(DEVICE) if meta_vals is not None else None

        meta_dim = (M_train.size(-1) if M_train is not None else 0)
        model = MultiOutputLSTMWithMeta(input_dim=1, output_dim=PREDICT, meta_dim=meta_dim).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train), device=DEVICE)
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch = X_train[batch_idx]
                y_batch = y_train[batch_idx]
                M_batch = M_train[batch_idx] if M_train is not None else None

                output = model(X_batch, M_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[key] = {
            'model': model.eval(),
            'sales_scaler': sales_scaler,
            'meta_scaler': meta_scaler,
            'meta_cols': meta_cols_all
        }

    return trained_models

In [ ]:
# 학습
trained_models = train_lstm_meta(train, TRAIN_META_DIR)
print("학습 완료. 모델 수:", len(trained_models))

# Prediction

In [ ]:
def make_meta_test(test_df, test_meta_dir, test_prefix, meta_cols_train):
    ski_t, room_t, hw_t, w_t = read_test_meta(test_meta_dir, test_prefix)

    merged = test_df.copy()
    merged["영업일자"] = pd.to_datetime(merged["영업일자"])

    def _merge_numeric_by_date(base, meta_df, allowed_cols=None):
        if meta_df is None or meta_df.empty:
            return base
        meta_df = meta_df.copy()
        meta_df["영업일자"] = pd.to_datetime(meta_df["영업일자"])
        if allowed_cols:
            num_cols = [c for c in allowed_cols if c in meta_df.columns]
        else:
            num_cols = [c for c in meta_df.columns if c != "영업일자" and np.issubdtype(meta_df[c].dtype, np.number)]
        if not num_cols:
            return base
        meta_df[num_cols] = meta_df[num_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
        return base.merge(meta_df[["영업일자"] + num_cols], on="영업일자", how="left")

    if META_OPTIONS["ski"]["use"]:
        merged = _merge_numeric_by_date(merged, ski_t, META_OPTIONS["ski"]["cols"])
    if META_OPTIONS["room"]["use"]:
        merged = _merge_numeric_by_date(merged, room_t, META_OPTIONS["room"]["cols"])
    if META_OPTIONS["hwadam"]["use"]:
        merged = _merge_numeric_by_date(merged, hw_t, META_OPTIONS["hwadam"]["cols"])
    if META_OPTIONS["weather"]["use"]:
        merged = _merge_numeric_by_date(merged, w_t, META_OPTIONS["weather"]["cols"])

    # price 병합
    price = None
    if META_OPTIONS.get("price", {}).get("use", False):
        _, _, _, _, price = read_train_meta(TRAIN_META_DIR, TRAIN_DIR)
    if price is not None and not price.empty and "영업장명_메뉴명" in price.columns:
        keep = ["영업장명_메뉴명"] + [c for c in (META_OPTIONS["price"]["cols"] or []) if c in price.columns]
        if len(keep) == 1:  # cols=None일 때는 숫자 전부
            keep = ["영업장명_메뉴명"] + [c for c in price.columns if c != "영업장명_메뉴명" and np.issubdtype(price[c].dtype, np.number)]
        merged = merged.merge(price[keep], on="영업장명_메뉴명", how="left")

    # 학습 때 사용한 meta 컬럼 목록을 가져옴
    meta_cols_used = list(meta_cols_train) if meta_cols_train else []
    if meta_cols_used:
        for c in meta_cols_used:
            if c not in merged.columns:
                merged[c] = 0.0
        merged[meta_cols_used] = merged[meta_cols_used].apply(pd.to_numeric, errors="coerce").fillna(0.0)

    item_to_seq = {}
    for item, g in merged.groupby("영업장명_메뉴명"):
        g = g.sort_values("영업일자").tail(LOOKBACK)
        X28 = g["매출수량"].values.astype(float).reshape(1, LOOKBACK, 1)
        M28 = g[meta_cols_used].values.astype(float).reshape(1, LOOKBACK, -1) if meta_cols_used else None
        item_to_seq[item] = (X28, M28)

    return item_to_seq, meta_cols_used

In [ ]:
def predict_lstm_meta(test_df, trained_models, test_prefix):
    results = []
    global TEST_META_DIR

    if not trained_models:
        return pd.DataFrame(columns=["영업일자","영업장명_메뉴명","매출수량"])

    any_key = next(iter(trained_models))
    meta_cols_train = trained_models[any_key]["meta_cols"]

    item_to_seq, meta_cols_used = make_meta_test(
        test_df=test_df,
        test_meta_dir=TEST_META_DIR,
        test_prefix=test_prefix,
        meta_cols_train=meta_cols_train,
    )

    for item in tqdm(test_df["영업장명_메뉴명"].unique(), desc=f"Predicting {test_prefix}", leave=True):
        if item not in trained_models or item not in item_to_seq:
            continue

        x28, m28 = item_to_seq[item]
        bundle   = trained_models[item]
        model    = bundle["model"]
        s_scaler = bundle["sales_scaler"]
        m_scaler = bundle["meta_scaler"]

        # 스케일 변환
        x28_s = s_scaler.transform(x28.reshape(-1, 1)).reshape(x28.shape)
        m28_s = None
        if (m28 is not None) and (m_scaler is not None):
            m28_s = m_scaler.transform(m28.reshape(-1, m28.shape[-1])).reshape(m28.shape)

        with torch.no_grad():
            x_t = torch.tensor(x28_s, dtype=torch.float32, device=DEVICE)
            m_t = torch.tensor(m28_s, dtype=torch.float32, device=DEVICE) if m28_s is not None else None
            pred_s = model(x_t, m_t).squeeze().cpu().numpy()  # (PREDICT,)

        # 역변환
        for i, d in enumerate([f"{test_prefix}+{k+1}일" for k in range(PREDICT)]):
            val = s_scaler.inverse_transform([[pred_s[i]]])[0, 0]
            results.append({
                "영업일자": d,
                "영업장명_메뉴명": item,
                "매출수량": max(val, 0.0)
            })

    return pd.DataFrame(results)


In [ ]:
# 모든 TEST_*.csv 순회
all_preds = []

test_files = sorted(glob.glob(os.path.join(TEST_DIR, "TEST_*.csv")))

for path in test_files:
    test_df = pd.read_csv(path, parse_dates=["영업일자"])
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)
    pred_df = predict_lstm_meta(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)